In [1]:
import numpy as np
import os
import librosa as lb
import soundfile as sf
import museval
from matplotlib import pyplot as plt
from tqdm import tqdm
import librosa.display

In [70]:
name = 'Music Delta - Hendrix/'
seconds = 0

clean_path = "/home/rajesh/Desktop/Datasets/musdb18hq/train/"+name

In [99]:
vocals, fs = lb.load(clean_path+'vocals.wav')
bass, fs = lb.load(clean_path+'bass.wav')
drums, fs = lb.load(clean_path+'drums.wav')
other, fs = lb.load(clean_path+'other.wav')

mixture, fs = lb.load(clean_path+'mixture.wav')

b1 = int(fs*seconds)
b2 = int(b1+(10*fs))

vocals = vocals[b1:b2]
bass = bass[b1:b2]
drums = drums[b1:b2]
other = other[b1:b2]
mixture = mixture[b1:b2]

In [100]:
def genbleed(vocals, bass, drums, other, way=1):
    
    if way == 1: # Normal with low bleed
        bvocals = 1 * vocals + 0.02 * bass + 0.09 * drums + 0.016 * other
        bbass = 0.021 * vocals + 1 * bass + 0.15 * drums + 0.16 * other
        bdrums = 0.01 * vocals + 0.12 * bass + 1 * drums + 0.16 * other
        bother = 0.09 * vocals + 0.02 * bass + 0.015 * drums + 1 * other
    if way == 2: # Diagonals varied with low bleed
        bvocals = 0.82 * vocals + 0.02 * bass + 0.09 * drums + 0.016 * other
        bbass = 0.021 * vocals + 0.71 * bass + 0.15 * drums + 0.16 * other
        bdrums = 0.01 * vocals + 0.12 * bass + 0.9 * drums + 0.16 * other
        bother = 0.09 * vocals + 0.02 * bass + 0.015 * drums + 0.66 * other
    if way == 3: # Normal with High Bleed
        bvocals = 1 * vocals + 0.2 * bass + 0.09 * drums + 0.16 * other
        bbass = 0.21 * vocals + 1 * bass + 0.15 * drums + 0.26 * other
        bdrums = 0.1 * vocals + 0.12 * bass + 1 * drums + 0.16 * other
        bother = 0.09 * vocals + 0.2 * bass + 0.15 * drums + 1 * other
    if way == 4: # Ideal high bleed
        bvocals = 1 * vocals + 0.2 * bass + 0.2 * drums + 0.2 * other
        bbass = 0.2 * vocals + 1 * bass + 0.2 * drums + 0.2 * other
        bdrums = 0.2 * vocals + 0.2 * bass + 1 * drums + 0.2 * other
        bother = 0.2 * vocals + 0.2 * bass + 0.2 * drums + 1 * other
    if way == 5: # Ideal low bleed
        bvocals = 1 * vocals + 0.05 * bass + 0.05 * drums + 0.05 * other
        bbass = 0.05 * vocals + 1 * bass + 0.05 * drums + 0.05 * other
        bdrums = 0.05 * vocals + 0.05 * bass + 1 * drums + 0.05 * other
        bother = 0.05 * vocals + 0.05 * bass + 0.05 * drums + 1 * other
        
    return bvocals, bbass, bdrums, bother

In [101]:
bvocals, bbass, bdrums, bother = genbleed(vocals, bass, drums, other, way=2)

In [102]:
smix = vocals+bass+drums+other
xmix = bvocals+bbass+bdrums+bother

In [103]:
np.linalg.norm(mixture-smix)**2, np.linalg.norm(mixture-xmix)**2

(0.00047804878076596405, 112.16503048359664)

In [104]:
S = np.array([vocals, bass, drums, other])

In [105]:
s_coeff = mixture @ np.linalg.pinv(S)
s_coeff

array([1.0000001, 1.0000002, 0.9999999, 0.999999 ], dtype=float32)

In [106]:
s_coeff = mixture @ np.linalg.pinv(S)
artmix_s_coeff = smix @ np.linalg.pinv(S)
artmix_x_coeff = xmix @ np.linalg.pinv(S)

np.round(s_coeff, 4), np.round(artmix_s_coeff, 4), np.round(artmix_x_coeff, 4)

(array([1., 1., 1., 1.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32),
 array([0.941, 0.87 , 1.155, 0.996], dtype=float32))

In [107]:
outpath = "/home/rajesh/Desktop/"

In [108]:
sf.write(outpath+"mixture.wav", mixture, fs)
sf.write(outpath+"smix.wav", smix, fs)
sf.write(outpath+"xmix.wav", xmix, fs)

In [109]:
sf.write(outpath+"bvocals.wav", bvocals, fs)
sf.write(outpath+"bbass.wav", bbass, fs)
sf.write(outpath+"bdrums.wav", bdrums, fs)
sf.write(outpath+"bother.wav", bother, fs)

In [110]:
sf.write(outpath+"vocals.wav", vocals, fs)
sf.write(outpath+"bass.wav", bass, fs)
sf.write(outpath+"drums.wav", drums, fs)
sf.write(outpath+"other.wav", other, fs)